# Task 1.1: TF–IDF + Logistic Regression Baseline

**Goal**: Establish a baseline for genre classification using TF–IDF features and Logistic Regression

**Expected performance**: Accuracy ~70-75%, Macro F1 ~0.68-0.73

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

## 2. Load and Explore Data

In [ ]:
# Load data
df = pd.read_csv('../data/cleaned_combined_guardian.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumn names:\n{df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

## 3. Analyze Categories

In [ ]:
# Check unique categories
print("Unique categories:")
print(df['category'].value_counts())
print(f"\nTotal unique categories: {df['category'].nunique()}")

In [ ]:
# Visualize category distribution
plt.figure(figsize=(10, 6))
df['category'].value_counts().plot(kind='bar')
plt.title('Category Distribution')
plt.xlabel('Category')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Check Missing Values and Text Length

In [ ]:
# Check for missing values
print("Missing values:")
print(df[['category', 'cleaned_text']].isnull().sum())

In [ ]:
# Check text length statistics
df['text_length'] = df['cleaned_text'].str.len()
print("Text length statistics:")
print(df['text_length'].describe())

In [ ]:
# Visualize text length distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df['text_length'], bins=50, edgecolor='black')
plt.title('Text Length Distribution')
plt.xlabel('Text Length (characters)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.boxplot(df['text_length'])
plt.title('Text Length Boxplot')
plt.ylabel('Text Length (characters)')

plt.tight_layout()
plt.show()

## 5. Data Cleaning

In [ ]:
# Remove rows with missing values
print(f"Original dataset size: {len(df)}")
df = df.dropna(subset=['category', 'cleaned_text'])
print(f"After removing NaN: {len(df)}")

In [ ]:
# Remove very short texts (< 50 characters)
df = df[df['cleaned_text'].str.len() > 50]
print(f"After removing short texts: {len(df)}")

In [ ]:
# Remove very long texts (> 20000 characters) - potential outliers
df = df[df['cleaned_text'].str.len() < 20000]
print(f"After removing very long texts: {len(df)}")

In [ ]:
# Check final category distribution
print("Final category distribution:")
print(df['category'].value_counts())

## 6. Prepare Data for Training

In [ ]:
# Extract texts and labels
texts = df['cleaned_text'].tolist()
labels = df['category'].tolist()

print(f"Total samples: {len(texts)}")
print(f"Labels: {set(labels)}")

## 7. Split Data (Train/Val/Test)

In [ ]:
# First split: 80% train, 20% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    texts, labels,
    test_size=0.2,
    stratify=labels,
    random_state=42
)

# Second split: 50% val, 50% test from temp (10% val, 10% test from total)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    stratify=y_temp,
    random_state=42
)

print(f"Training set: {len(X_train)} samples")
print(f"Validation set: {len(X_val)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"\nTotal: {len(X_train) + len(X_val) + len(X_test)} samples")

In [ ]:
# Check label distribution in splits
print("\nTraining label distribution:")
print(Counter(y_train))
print("\nValidation label distribution:")
print(Counter(y_val))
print("\nTest label distribution:")
print(Counter(y_test))

## 8. Save Data Splits

In [ ]:
# Save splits for reuse
splits = {
    'X_train': X_train,
    'X_val': X_val,
    'X_test': X_test,
    'y_train': y_train,
    'y_val': y_val,
    'y_test': y_test
}

joblib.dump(splits, '../results/data_splits.pkl')
print("Data splits saved to ../results/data_splits.pkl")

## 9. TF–IDF Vectorization

In [ ]:
# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer(
    max_features=10000,      # Top 10k words
    ngram_range=(1, 2),      # Unigrams + bigrams
    min_df=5,                # Min 5 documents
    max_df=0.8,              # Max 80% documents
    stop_words='english'     # Remove stop words
)

print("TF–IDF vectorizer initialized")

In [ ]:
# Fit on training data and transform all sets
print("Fitting vectorizer on training data...")
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

print(f"Training set shape: {X_train_tfidf.shape}")
print(f"Validation set shape: {X_val_tfidf.shape}")
print(f"Test set shape: {X_test_tfidf.shape}")

## 10. Grid Search for Logistic Regression

In [ ]:
# Define parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization strength
    'penalty': ['l2'],        # L2 regularization
    'solver': ['lbfgs']       # Solver for multinomial
}

# Initialize logistic regression
lr_base = LogisticRegression(
    max_iter=1000,
    random_state=42,
    multi_class='multinomial'
)

print("Starting Grid Search...")
print(f"Parameter grid: {param_grid}")

In [ ]:
# Perform grid search with 5-fold CV
grid_search = GridSearchCV(
    lr_base,
    param_grid,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_tfidf, y_train)

In [ ]:
# Best parameters and score
print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

## 11. Train Final Model

In [ ]:
# Get best model
best_model = grid_search.best_estimator_

# Predictions
y_train_pred = best_model.predict(X_train_tfidf)
y_val_pred = best_model.predict(X_val_tfidf)
y_test_pred = best_model.predict(X_test_tfidf)

print("Predictions completed!")

## 12. Evaluate Model

In [ ]:
# Training set performance
train_acc = accuracy_score(y_train, y_train_pred)
train_f1_macro = f1_score(y_train, y_train_pred, average='macro')

print("=== Training Set ===")
print(f"Accuracy: {train_acc:.4f}")
print(f"Macro F1: {train_f1_macro:.4f}")

In [ ]:
# Validation set performance
val_acc = accuracy_score(y_val, y_val_pred)
val_f1_macro = f1_score(y_val, y_val_pred, average='macro')

print("=== Validation Set ===")
print(f"Accuracy: {val_acc:.4f}")
print(f"Macro F1: {val_f1_macro:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred))

In [ ]:
# Test set performance
test_acc = accuracy_score(y_test, y_test_pred)
test_f1_macro = f1_score(y_test, y_test_pred, average='macro')

print("=== Test Set ===")
print(f"Accuracy: {test_acc:.4f}")
print(f"Macro F1: {test_f1_macro:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

## 13. Confusion Matrix

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Get class labels
class_labels = best_model.classes_

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.title('Confusion Matrix - Test Set')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.savefig('../results/tfidf_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("Confusion matrix saved to ../results/tfidf_confusion_matrix.png")

## 14. Extract Lexical Markers for Each Genre

In [ ]:
# Get feature names
feature_names = vectorizer.get_feature_names_out()

print(f"Total features: {len(feature_names)}")
print(f"\nClass labels: {class_labels}")

In [ ]:
# Extract top words for each genre
lexical_markers = {}

for i, genre in enumerate(class_labels):
    # Get coefficients for this class
    coef = best_model.coef_[i]
    
    # Get top 50 words with highest weights
    top_indices = np.argsort(coef)[-50:][::-1]
    
    # Store word-weight pairs
    lexical_markers[genre] = [
        (feature_names[idx], coef[idx])
        for idx in top_indices
    ]

print("Lexical markers extracted for all genres!")

In [ ]:
# Display top 20 words for each genre
for genre, markers in lexical_markers.items():
    print(f"\n{'='*60}")
    print(f"GENRE: {genre.upper()}")
    print(f"{'='*60}")
    for word, weight in markers[:20]:
        print(f"{word:25s} {weight:8.4f}")

## 15. Save Model and Results

In [ ]:
# Save vectorizer
joblib.dump(vectorizer, '../models/tfidf_vectorizer.pkl')
print("✓ Vectorizer saved to ../models/tfidf_vectorizer.pkl")

In [ ]:
# Save model
joblib.dump(best_model, '../models/tfidf_lr.pkl')
print("✓ Model saved to ../models/tfidf_lr.pkl")

In [ ]:
# Save confusion matrix
joblib.dump(cm, '../results/tfidf_confusion_matrix.npy')
print("✓ Confusion matrix saved to ../results/tfidf_confusion_matrix.npy")

In [ ]:
# Save lexical markers
joblib.dump(lexical_markers, '../results/tfidf_lexical_markers.pkl')
print("✓ Lexical markers saved to ../results/tfidf_lexical_markers.pkl")

In [ ]:
# Save metrics
metrics = {
    'model': 'TF–IDF + Logistic Regression',
    'best_params': grid_search.best_params_,
    'train_accuracy': float(train_acc),
    'train_f1_macro': float(train_f1_macro),
    'val_accuracy': float(val_acc),
    'val_f1_macro': float(val_f1_macro),
    'test_accuracy': float(test_acc),
    'test_f1_macro': float(test_f1_macro),
    'cv_score': float(grid_search.best_score_)
}

import json
with open('../results/tfidf_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("✓ Metrics saved to ../results/tfidf_metrics.json")

In [ ]:
# Display final results
print("\n" + "="*60)
print("TF–IDF + LOGISTIC REGRESSION - FINAL RESULTS")
print("="*60)
print(f"\nBest Parameters: {grid_search.best_params_}")
print(f"\nCV Score (F1-macro): {grid_search.best_score_:.4f}")
print(f"\n--- Test Set Performance ---")
print(f"Accuracy: {test_acc:.4f}")
print(f"Macro F1: {test_f1_macro:.4f}")
print(f"\n--- Expected vs Actual ---")
print(f"Expected Accuracy: ~70-75%")
print(f"Actual Accuracy: {test_acc:.2%}")
print(f"\nExpected Macro F1: ~0.68-0.73")
print(f"Actual Macro F1: {test_f1_macro:.4f}")
print("="*60)

## 16. Summary

### ✅ Completed:
1. Loaded and explored data (50K texts from The Guardian)
2. Cleaned data (removed NaN, very short/long texts)
3. Split data into train/val/test (80/10/10)
4. Created TF–IDF vectors (10k features, unigrams + bigrams)
5. Performed Grid Search for Logistic Regression
6. Extracted lexical markers for each genre (top 50 words)
7. Saved model, vectorizer, and results

### 📊 Results:
- Test Accuracy: **{:.2%}".format(test_acc))
- Test Macro F1: **{:.4f}".format(test_f1_macro))
- Confusion matrix saved
- Lexical markers for each genre extracted

### 🎯 Next Steps:
1. Analyze confusion matrix for error patterns
2. Compare with linguistic features model
3. Compare with BERT/RoBERTa

### 📁 Files Created:
- `../models/tfidf_vectorizer.pkl` - TF–IDF vectorizer
- `../models/tfidf_lr.pkl` - Trained Logistic Regression model
- `../results/tfidf_metrics.json` - Performance metrics
- `../results/tfidf_confusion_matrix.npy` - Confusion matrix
- `../results/tfidf_confusion_matrix.png` - Confusion matrix visualization
- `../results/tfidf_lexical_markers.pkl` - Top words for each genre
- `../results/data_splits.pkl` - Train/val/test splits